In [86]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [87]:
#data = pd.read_excel("D:\MAGVI_500RT_RawData.xlsx")
url = "https://github.com/dklim77/MechLearnEdu/blob/master/MAGVI_500RT_RawData.xlsx?raw=true"
data = pd.read_excel(url)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1524 entries, 0 to 1523
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   No                  1524 non-null   int64  
 1   EvaporatingTemp     1524 non-null   float64
 2   CondensingTemp      1524 non-null   float64
 3   IGVOpening          1524 non-null   int64  
 4   HotGasFraction      1524 non-null   int64  
 5   RPM                 1524 non-null   int64  
 6   Capacity            1524 non-null   float64
 7   InputPower          1524 non-null   float64
 8   FirstTopHead        1524 non-null   float64
 9   SecondTopHead       1524 non-null   float64
 10  1stCompHead         1524 non-null   float64
 11  2ndCompHead         1524 non-null   float64
 12  TotalHead           1524 non-null   float64
 13  1stCompSurgeMargin  1524 non-null   float64
 14  2ndCompSurgeMargin  1524 non-null   float64
 15  VFR_1st             1524 non-null   float64
 16  VFR_2n

In [88]:
print(data.shape)
data[:5]

(1524, 17)


,No,EvaporatingTemp,CondensingTemp,IGVOpening,HotGasFraction,RPM,Capacity,InputPower,FirstTopHead,SecondTopHead,1stCompHead,2ndCompHead,TotalHead,1stCompSurgeMargin,2ndCompSurgeMargin,VFR_1st,VFR_2nd
0,2,2.0,19.0,80,0,15001,543.8,274.61,1664.8,1155.8,1072.7,136.3,1209.0,592.1,1019.5,0.7060,0.4532
1,3,2.0,19.0,60,0,15001,479.5,224.12,1585.0,1100.4,1016.9,192.4,1209.4,568.1,907.9,0.6183,0.4088
2,4,2.0,19.0,40,0,15001,389.1,186.39,1550.7,1077.4,979.7,234.1,1213.8,571.0,843.3,0.4995,0.3392
3,5,2.0,19.0,20,0,15001,254.7,146.55,1512.2,1053.8,931.7,300.5,1232.1,580.5,753.4,0.3253,0.2330
4,6,2.0,19.0,0,0,15001,58.3,94.05,1095.8,771.2,1060.0,292.3,1352.3,35.8,478.9,0.0760,0.0606


In [89]:
# 사본 사용 (원본 백업)
df = data.copy()
df[:3]

,No,EvaporatingTemp,CondensingTemp,IGVOpening,HotGasFraction,RPM,Capacity,InputPower,FirstTopHead,SecondTopHead,1stCompHead,2ndCompHead,TotalHead,1stCompSurgeMargin,2ndCompSurgeMargin,VFR_1st,VFR_2nd
0,2,2.0,19.0,80,0,15001,543.8,274.61,1664.8,1155.8,1072.7,136.3,1209.0,592.1,1019.5,0.7060,0.4532
1,3,2.0,19.0,60,0,15001,479.5,224.12,1585.0,1100.4,1016.9,192.4,1209.4,568.1,907.9,0.6183,0.4088
2,4,2.0,19.0,40,0,15001,389.1,186.39,1550.7,1077.4,979.7,234.1,1213.8,571.0,843.3,0.4995,0.3392


In [90]:
# 스케일링

# 표준 스케일링
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
df[["EvaporatingTemp_sc","CondensingTemp_sc","IGVOpening_sc","RPM_sc"]] = scale.fit_transform(df[["EvaporatingTemp","CondensingTemp","IGVOpening","RPM"]])

# Min Max 스케일링
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
df[["EvaporatingTemp_mmsc","CondensingTemp_mmsc","IGVOpening_mmsc","RPM_mmsc"]] = minmax.fit_transform(df[["EvaporatingTemp","CondensingTemp","IGVOpening","RPM"]])

df[:3]

,No,EvaporatingTemp,CondensingTemp,IGVOpening,HotGasFraction,RPM,Capacity,InputPower,FirstTopHead,SecondTopHead,1stCompHead,2ndCompHead,TotalHead,1stCompSurgeMargin,2ndCompSurgeMargin,VFR_1st,VFR_2nd,EvaporatingTemp_sc,CondensingTemp_sc,IGVOpening_sc,RPM_sc,EvaporatingTemp_mmsc,CondensingTemp_mmsc,IGVOpening_mmsc,RPM_mmsc
0,2,2.0,19.0,80,0,15001,543.8,274.61,1664.8,1155.8,1072.7,136.3,1209.0,592.1,1019.5,0.7060,0.4532,-1.622635,-1.391129,0.668366,1.095127,0.0,0.0,0.8,1.0
1,3,2.0,19.0,60,0,15001,479.5,224.12,1585.0,1100.4,1016.9,192.4,1209.4,568.1,907.9,0.6183,0.4088,-1.622635,-1.391129,-0.019406,1.095127,0.0,0.0,0.6,1.0
2,4,2.0,19.0,40,0,15001,389.1,186.39,1550.7,1077.4,979.7,234.1,1213.8,571.0,843.3,0.4995,0.3392,-1.622635,-1.391129,-0.707177,1.095127,0.0,0.0,0.4,1.0


In [91]:
#X = df[["EvaporatingTemp", "CondensingTemp", "IGVOpening", "RPM"]]
# 표준 스케일링 적용
X = df[["EvaporatingTemp_sc","CondensingTemp_sc","IGVOpening_sc","RPM_sc"]]

# Min Max 스케일링 적용
#X = df[["EvaporatingTemp_mmsc","CondensingTemp_mmsc","IGVOpening_mmsc","RPM_mmsc"]]

y = df[["Capacity", "InputPower", "FirstTopHead", "SecondTopHead", "1stCompHead", "2ndCompHead"]]
X[:5], y[:5]


(   EvaporatingTemp_sc  CondensingTemp_sc  IGVOpening_sc    RPM_sc
 0           -1.622635          -1.391129       0.668366  1.095127
 1           -1.622635          -1.391129      -0.019406  1.095127
 2           -1.622635          -1.391129      -0.707177  1.095127
 3           -1.622635          -1.391129      -1.394948  1.095127
 4           -1.622635          -1.391129      -2.082720  1.095127,
    Capacity  InputPower  FirstTopHead  SecondTopHead  1stCompHead  2ndCompHead
 0     543.8      274.61        1664.8         1155.8       1072.7        136.3
 1     479.5      224.12        1585.0         1100.4       1016.9        192.4
 2     389.1      186.39        1550.7         1077.4        979.7        234.1
 3     254.7      146.55        1512.2         1053.8        931.7        300.5
 4      58.3       94.05        1095.8          771.2       1060.0        292.3)

In [92]:
from sklearn.model_selection import train_test_split
from keras import layers
from keras import models
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [93]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
# monitor='val_loss' : validation set 의 loss 를 monitoring 한다
# mode='min' : performance measure를 최소화 시킴, default는 'auto'
# verbose=1 : 언제 keras 에서 training 을 멈추었는지를 화면에 출력

# mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='min', save_best_only=True)
# Early stopping 객체에 의해 트레이닝이 중지되었을 때 그 상태는 이전 모ㅔㄹ에 비해 일반적으로
# validation error가 높은 상이일 것이다. 따라사 중지된 상태가 최고의 모델이 아닐 것이다.
# 가장 validation performance가 좋은 모델을 저장하기 위해 keras에서는 ModelCheckpoint 객체가 존재

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=9)    ################
model = models.Sequential()
model.add(layers.Dense(70, activation='relu', input_shape = (4,)))    # 스케일링 할 것  x
#model.add(layers.BatchNormalization())
model.add(layers.Dense(70, activation='relu'))
#model.add(layers.Dense(100, activation='relu'))
# model.add(layers.Dense(100, activation='relu'))
#model.add(layers.BatchNormalization())
#model.add(layers.Dropout(0.2))
model.add(layers.Dense(6, activation='linear'))    # activation='linear'  디폴트값임
model.summary()
# 안되면 레이어 추가할 것

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 70)                350       
_________________________________________________________________
dense_13 (Dense)             (None, 70)                4970      
_________________________________________________________________
dense_14 (Dense)             (None, 6)                 426       
Total params: 5,746
Trainable params: 5,746
Non-trainable params: 0
_________________________________________________________________


In [95]:
model.compile(optimizer= 'adam', loss = 'mean_squared_error')
history = model.fit(X_train, y_train, epochs=300, batch_size=20, verbose=1, validation_split = 0.2, callbacks=[early_stopping])    # early stopping   5정도 적용  5~10

Epoch 1/300
43/43 [==============================] - 1s 8ms/step - loss: 651958.0014 - val_loss: 652105.2500
Epoch 2/300
43/43 [==============================] - 0s 2ms/step - loss: 664045.8878 - val_loss: 640646.8750
Epoch 3/300
43/43 [==============================] - 0s 2ms/step - loss: 638047.4162 - val_loss: 606020.5000
Epoch 4/300
43/43 [==============================] - 0s 2ms/step - loss: 590279.2543 - val_loss: 534320.8125
Epoch 5/300
43/43 [==============================] - 0s 2ms/step - loss: 500549.2656 - val_loss: 424805.2812
Epoch 6/300
43/43 [==============================] - 0s 2ms/step - loss: 390200.1286 - val_loss: 299929.4688
Epoch 7/300
43/43 [==============================] - 0s 2ms/step - loss: 264621.9091 - val_loss: 189347.9844
Epoch 8/300
43/43 [==============================] - 0s 2ms/step - loss: 161694.2209 - val_loss: 112614.9688
Epoch 9/300
43/43 [==============================] - 0s 3ms/step - loss: 96455.7534 - val_loss: 68517.6875
Epoch 10/300
43/43 [=

In [96]:
X_test.shape

(458, 4)

In [97]:
model.predict(X_test[:10])

array([[ 474.4767 ,  250.25938, 1255.8457 ,  880.4093 , 1194.2603 ,
         789.68585],
       [ 537.6549 ,  239.47374, 1250.9512 ,  878.29474, 1041.4608 ,
         522.6125 ],
       [ 165.53528,  183.99875, 1520.8156 , 1062.1396 , 1449.7542 ,
         993.86725],
       [ 483.69864,  287.58405, 1421.8846 ,  992.1325 , 1276.8752 ,
         830.7962 ],
       [ 433.72726,  182.37372, 1352.6035 ,  949.24695,  942.8351 ,
         209.50462],
       [ 642.9681 ,  346.68356, 1733.6257 , 1209.1189 , 1149.208  ,
         268.7492 ],
       [ 115.99806,  128.44533, 1270.2354 ,  876.4655 ,  977.0773 ,
         338.43018],
       [ 365.66495,  133.72269, 1098.169  ,  770.7686 ,  820.1865 ,
         256.9381 ],
       [ 482.1333 ,  236.1432 , 1358.761  ,  951.2519 , 1051.8815 ,
         506.04102],
       [ 289.95813,  192.91263, 1486.5809 , 1041.9633 , 1039.3801 ,
         480.27142]], dtype=float32)

In [98]:
y_test.shape

(458, 6)

In [99]:
import pandas as pd
y_pred = model.predict(X_test)     # ravel 삭제

In [100]:
y_pred.shape

(458, 6)

In [101]:
y_test.shape

(458, 6)

In [102]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)     # 학습 부족    # 학습 과대 

0.9936764079637888

In [103]:
#model.get_weights()

#model.get_weights()[2].shape

weights = model.get_weights()
weights
#model.set_weights(weights)

print(weights[0].shape)
print(weights[1].shape)
print(weights[2].shape)
print(weights[3].shape)
print(weights[4].shape)
print(weights[5].shape)


(4, 70)
(70,)
(70, 70)
(70,)
(70, 6)
(6,)


In [105]:
np.savetxt('weights[0].csv', weights[0])
np.savetxt('weights[1].csv', weights[1])
np.savetxt('weights[2].csv', weights[2])
np.savetxt('weights[3].csv', weights[3])
np.savetxt('weights[4].csv', weights[4])
np.savetxt('weights[5].csv', weights[5])
